In [1]:
import torch
cuda_avail = torch.cuda.is_available()
device = torch.device("cuda" if cuda_avail else "cpu")
cerealsModel = torch.load("./trained/tasks/cereals_epoch20_2020-04-07.pkl", map_location=torch.device(device))
coffeeModel = torch.load("./trained/tasks/coffee_epoch30_2020-04-07.pkl", map_location=torch.device(device))
friedeggModel = torch.load("./trained/tasks/friedegg_epoch30_2020-04-07.pkl", map_location=torch.device(device))
juiceModel = torch.load("./trained/tasks/juice_epoch40_2020-04-07.pkl", map_location=torch.device(device))
milkModel = torch.load("./trained/tasks/milk_epoch40_2020-04-07.pkl", map_location=torch.device(device))
pancakeModel = torch.load("./trained/tasks/pancake_epoch40_2020-04-07.pkl", map_location=torch.device(device))
salatModel = torch.load("./trained/tasks/salat_epoch30_2020-04-07.pkl", map_location=torch.device(device))
sandwichModel = torch.load("./trained/tasks/sandwich_epoch10_2020-04-07.pkl", map_location=torch.device(device))
scrambledeggModel = torch.load("./trained/tasks/scrambledegg_epoch40_2020-04-07.pkl", map_location=torch.device(device))
teaModel = torch.load("./trained/tasks/tea_epoch10_2020-04-07.pkl", map_location=torch.device(device))

modelDictionary = {
    "cereals" : cerealsModel,
    "coffee" : coffeeModel,
    "friedegg" : friedeggModel,
    "juice" : juiceModel,
    "milk" : milkModel,
    "pancake" : pancakeModel,
    "salat" : salatModel,
    "sandwich" : sandwichModel,
    "scrambledegg" : scrambledeggModel,
    "tea" : teaModel
}

In [2]:
import pickle
def loadDict(filename):
    pkl_file = open(filename, 'rb')
    dict = pickle.load(pkl_file)
    pkl_file.close()
    return dict

cerealsMapping = loadDict("./trained/tasks/dictionary/cereals.pkl")
coffeeMapping = loadDict("./trained/tasks/dictionary/coffee.pkl")
friedeggMapping = loadDict("./trained/tasks/dictionary/friedegg.pkl")
juiceMapping = loadDict("./trained/tasks/dictionary/juice.pkl")
milkMapping = loadDict("./trained/tasks/dictionary/milk.pkl")
pancakeMapping = loadDict("./trained/tasks/dictionary/pancake.pkl")
salatMapping = loadDict("./trained/tasks/dictionary/salat.pkl")
sandwichMapping = loadDict("./trained/tasks/dictionary/sandwich.pkl")
scrambledeggMapping = loadDict("./trained/tasks/dictionary/scrambledegg.pkl")
teaMapping = loadDict("./trained/tasks/dictionary/tea.pkl")

mappingDictionary = {
    "cereals" : cerealsMapping,
    "coffee" : coffeeMapping,
    "friedegg" : friedeggMapping,
    "juice" : juiceMapping,
    "milk" : milkMapping,
    "pancake" : pancakeMapping,
    "salat" : salatMapping,
    "sandwich" : sandwichMapping,
    "scrambledegg" : scrambledeggMapping,
    "tea" : teaMapping
}

In [3]:
from utils.read_datasetBreakfast import load_testdata
import os

COMP_PATH = ''

test_split  =  os.path.join(COMP_PATH, 'splits/test.split1.bundle') #Test Split
DATA_folder =  os.path.join(COMP_PATH, 'Data/') #Frame I3D features for all videos
data_feat = load_testdata(test_split, DATA_folder)

Finish Load the Test data!!!


In [4]:
segment_idx = []
with open("./utils/test_segment.txt") as f:
    while True:
        line = f.readline()
        if not line:
            break
        segment_idx.append(line[:-1].split(" "))

In [5]:
data_feat_seg = []
for i in range(len(data_feat)):
    videoFrames = []
    for j in range(len(segment_idx[i])-1):
        videoFrames.append(data_feat[i][1][int(segment_idx[i][j]):int(segment_idx[i][j+1])])
    data_feat_seg.append((data_feat[i][0], videoFrames))

In [10]:
def get_max_prob_seg_seq(outputs, prob_mat, device):
    # 48
    action_nums = prob_mat.shape[0]
    f = torch.zeros(len(outputs), action_nums).to(device)
    record = torch.zeros(len(outputs), action_nums).to(device).long()

    for i, i_prob in enumerate(outputs):
        if i == 0:
            f[i] = i_prob
        else:
            for j in range(action_nums):
                last_to_j_max_prob, last_j = torch.max(f[i-1] + torch.log(prob_mat[:, j]), 0)
                f[i][j] = last_to_j_max_prob + i_prob[j]
                # print(last_to_j_max_prob, i_prob[j])
                record[i][j] = last_j
        # f[i] = normalize(f[i],dim=0)

    # get final max prob
    max_final_prob, max_final_action = torch.max(f[len(outputs)-1], 0)

    # get max prob seq
    seq = [max_final_action.item()]
    for k in reversed(range(1, len(outputs))):
        last_action = record[k][max_final_action]
        seq.append(last_action.item())
        max_final_action = last_action
    
    seq.reverse()
    return seq


In [11]:
import numpy as np
from Dataset.SegmentDataset import SegmentDataset
import torch.utils.data as tud

batch_size = 1
outputs = []
prob_mat = torch.load('./trained/conv/prob_mat_next.pt', map_location=torch.device(device))
for i in data_feat_seg:
    task = i[0]
    fake_labels = np.ones(len(i[1]))
    test_dataset = SegmentDataset(i[1], fake_labels, seed = 2)
    test_dataloader = tud.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    labelMapping = mappingDictionary[task]
    label_nums = len(labelMapping)
    model = modelDictionary[task].to(device)
    model.eval()
    seg_labels = []
    for x, labels in test_dataloader:
        x = x.to(device)
        with torch.no_grad():
            output = model(x).to(device)
            output = output.view(-1, label_nums)
            predict_label = torch.max(output, 1)[1]
            original_label = labelMapping[predict_label.item()]
            #print(task +" : "+ str(original_label))
            seg_labels.append(original_label)
    new_labels = get_max_prob_seg_seq(seg_labels, prob_mat, device)
    print(task + " : " + str(new_labels))
    outputs += new_labels
    

TypeError: 'int' object is not subscriptable

In [41]:
print(len(outputs))
            

1284


In [43]:
path = './results/tasks/test_result_bytask.csv'
f = open(path, 'w+')
f.write('Id,Category\n')

counter = 0
for i in outputs:
        f.write(f'{counter},{i}\n')
        counter += 1
f.close()

